<a href="https://colab.research.google.com/github/christeeno/mljourney/blob/main/patent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ziya07/student-health-data")

print("Path to dataset files:", path)

100%|██████████| 70.8k/70.8k [00:00<00:00, 49.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/ziya07/student-health-data/versions/1


In [ ]:
import os
import pandas as pd

data=pd.read_csv(os.path.join(path, 'student_health_data.csv'))
data.head()

,Student_ID,Age,Gender,Heart_Rate,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,Stress_Level_Biosensor,Stress_Level_Self_Report,Physical_Activity,Sleep_Quality,Mood,Study_Hours,Project_Hours,Health_Risk_Level
0,1,24,M,50.663217,122.173015,84.419860,3.137350,9.028669,High,Moderate,Happy,34.520973,16.800956,Moderate
1,2,21,F,57.926042,110.778407,75.696145,3.699078,5.819697,Moderate,Good,Stressed,16.763846,15.791154,Moderate
2,3,22,M,59.294219,109.375673,83.803814,6.785156,5.892360,Low,Moderate,Happy,44.203798,25.678437,Moderate
3,4,24,M,76.826232,125.142227,78.091587,6.408509,6.884001,High,Poor,Happy,21.776645,20.808391,High
4,5,20,M,68.342769,107.515592,80.674937,7.264719,4.483450,Moderate,Poor,Happy,8.964999,15.194045,Moderate


In [ ]:
data.dtypes

,0
Student_ID,int64
Age,int64
Gender,object
Heart_Rate,float64
Blood_Pressure_Systolic,float64
Blood_Pressure_Diastolic,float64
Stress_Level_Biosensor,float64
Stress_Level_Self_Report,float64
Physical_Activity,object
Sleep_Quality,object


In [ ]:
data.isnull().sum()

,0
Student_ID,0
Age,0
Gender,0
Heart_Rate,0
Blood_Pressure_Systolic,0
Blood_Pressure_Diastolic,0
Stress_Level_Biosensor,0
Stress_Level_Self_Report,0
Physical_Activity,0
Sleep_Quality,0


In [ ]:
mask = pd.Series(True, index=data.index)
for col in data.columns:
    if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Update mask to exclude outliers
        mask &= (data[col] >= lower_bound) & (data[col] <= upper_bound)

# Apply mask to filter the entire DataFrame
data_cleaned = data[mask]

In [ ]:
data1 = data_cleaned.copy()

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
for col in data1.columns:
  if data1[col].dtype=='object':
    data1[col]=label_encoder.fit_transform(data1[col])

In [ ]:
data1.dtypes

,0
Student_ID,int64
Age,int64
Gender,int64
Heart_Rate,float64
Blood_Pressure_Systolic,float64
Blood_Pressure_Diastolic,float64
Stress_Level_Biosensor,float64
Stress_Level_Self_Report,float64
Physical_Activity,int64
Sleep_Quality,int64


In [ ]:
from sklearn.model_selection import train_test_split
y=data1.iloc[:,-1]
X=data1.drop(columns=['Student_ID','Health_Risk_Level'])
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
model=LogisticRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.8316326530612245
Confusion Matrix:
 [[ 16   2  14]
 [  0  24  10]
 [  6   1 123]]
Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.50      0.59        32
           1       0.89      0.71      0.79        34
           2       0.84      0.95      0.89       130

    accuracy                           0.83       196
   macro avg       0.82      0.72      0.76       196
weighted avg       0.83      0.83      0.82       196



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9795918367346939

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.88      0.93        32
           1       1.00      1.00      1.00        34
           2       0.97      1.00      0.98       130

    accuracy                           0.98       196
   macro avg       0.99      0.96      0.97       196
weighted avg       0.98      0.98      0.98       196



In [ ]:
import joblib

# Save the model to a file
randomforest = 'random_forest_model.joblib'
joblib.dump(model, randomforest)

print(f"Random Forest model saved to {randomforest}")

Random Forest model saved to random_forest_model.joblib


**Reasoning**:
Load the saved random forest model using joblib.

